<a href="https://colab.research.google.com/github/omk42/Pattern-Mining/blob/master/CNNFeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installations**


Caffe GPU

In [ ]:
!apt install caffe-cuda

Caffe CPU

In [ ]:
!apt install caffe-cpu

**Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
from skimage.util.shape import view_as_windows
from skimage.transform import resize
import scipy.io as sio
import pickle

In [ ]:
import caffe

**Configurations**

In [ ]:
ROOT = '/content/drive/My Drive/MDPM/'

conf_dataset = 'shoes_trim'
conf_imgDir = ROOT + 'data/' + conf_dataset
conf_numClasses = 1
conf_numSamples = 100

# conf_patchSize = 128
# conf_patchSizeL2 = 160
# conf_patchSizeL3 = 192
conf_stepSize = 24
conf_patchSize = 48

# conf_caffe_def = ROOT + "cnn/deploy1_fc6.prototxt"
conf_caffe_def_fc7 = ROOT + "cnn/deploy1_fc7.prototxt"
conf_caffe_model = ROOT + "cnn/bvlc_reference_caffenet.caffemodel"

conf_numTopActivation = 20
conf_supp = 0.01
conf_minLength = 3
conf_maxLength = 6
conf_confid = 30

CROPPED_DIM = 227
IMAGE_DIM = 256

conf_output_dir = ROOT+ 'output/' + conf_dataset + '_fc7'
conf_image_mean = conf_output_dir + '/dataset_mean.npy'


# conf_cnnFeatures = ROOT+'output/' + conf_dataset + '/cnnFeatures2.npy'
# conf_imageDict = ROOT + 'output/' + conf_dataset + '/imageDict.pkl'
# conf_indexImg = ROOT + 'output/' + conf_dataset + '/indexImg.pkl'

conf_cnnFeatures = conf_output_dir + '/cnnFeatures.npy'
conf_imageDict = conf_output_dir + '/imageDict.pkl'
conf_indexImg = conf_output_dir + '/indexImg.pkl'
# conf_imageDict_new = conf_output_dir + '/imageDict5.pkl'
# conf_indexImg_new = conf_output_dir + '/indexImg5.pkl'

In [ ]:
IMAGE_MEAN = np.load(conf_image_mean)
labels = [x for x in Path(conf_imgDir).iterdir()]

In [ ]:
def setup_caffe():
  caffe.set_mode_cpu()
  # net = caffe.Net(conf_caffe_def, conf_caffe_model, caffe.TEST)
  net = caffe.Net(conf_caffe_def_fc7, conf_caffe_model, caffe.TEST)  
  return net

In [ ]:
def caffe_net_forward (patch, net):
  #caffe.set_device(0)
  #caffe.set_mode_gpu()
  transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
  transformer.set_mean('data', IMAGE_MEAN)
  transformer.set_transpose('data', (2,0,1))
  transformer.set_channel_swap('data', (2,1,0))
  #transformer.set_raw_scale('data', 255.0)
  net.blobs['data'].reshape(1,3,CROPPED_DIM, CROPPED_DIM)
  net.blobs['data'].data[...] = transformer.preprocess('data', patch)
  #print (net.blobs['data'].data.shape, net.blobs['data'].data)
  #test= net.blobs['data'].data[0]
  #debug_img((test.transpose(1,2,0)+256)/2, "Transformed patch" + str(np.all(test<256)))
  #debug_img(transformer.deprocess('data',net.blobs['data'].data[0]), "DeTransformed patch"+ str(np.all(transformer.deprocess('data',net.blobs['data'].data[0])<256)))
  #return 
  out = net.forward()
  # print ("Shape of output ", out['fc6'].shape)
  return out['fc7']

In [ ]:
def cnnFeaExtraction(classId, patchSize):
  label = labels[classId-1]  
  #print ("Shape of IMAGE MEAN is ", IMAGE_MEAN.shape)
  image_dict = dict()
  indexImage = list()
  net = setup_caffe()

  # with open(conf_imageDict, 'rb') as imageDictFile, open(conf_indexImg, 'rb') as indexImgFile:
  #   # imageDictFile.seek(0)
  #   # indexImgFile.seek(0)
  #   image_dict = pickle.load(imageDictFile)
  #   print ("Image Dict is ", len(image_dict))
  #   indexImage = pickle.load(indexImgFile) 
  #   print ("Index Img is ", len(indexImage))

  for imageID, im_path in enumerate(label.iterdir()):
    if imageID in image_dict:
      if image_dict[imageID] != im_path:
        print (image_dict[imageID], im_path)
      assert (image_dict[imageID] == im_path)
      continue
    print ("Finished images=  ", imageID) 
    # if imageID >= 4000:
    #   break
    image_dict[imageID] = im_path
    
    patches = sample_images(im_path, patchSize)
    #print("Shape of patches is ", patches.shape)
    # num_patches = patches.shape[0]*patches.shape[1] 
    # indexImage += [imageID for _ in range(num_patches)]

    # images = np.zeros((CROPPED_DIM, CROPPED_DIM, 3, num_patches), dtype=float)
    # cnnFea = np.zeros((4096, num_patches))
    #print ("Shape of images is ", images.shape)
    #print ("Shape of cnnFea is ", cnnFea.shape)

    patchID = 0
    for i in range(patches.shape[0]):
      for j in range(patches.shape[1]):
        patch = patches[i][j][0]
        #patch = resize(patch, (IMAGE_DIM, IMAGE_DIM))
        patch = resize(patch, (CROPPED_DIM, CROPPED_DIM))
        patch = np.swapaxes(patch, 0, 1)
        # images[:,:,:,patchID] = patch
        #debug_img (patch, "OG patch looks like"+ str(np.all(patch<256)))
        
        patch_flat = np.sum(patch, axis=2)
        # print (pch.size, np.nonzero(pch-255)[0].size, int(0.2*pch.size))
        if np.nonzero(patch_flat-765.0)[0].size > int(0.2*patch_flat.size):
          out = caffe_net_forward(patch, net)
          # print (out.shape)
          if patchID == 0:
            cnnFea = out.T
          else:
            cnnFea = np.concatenate((cnnFea, out.T), axis=1)
          patchID += 1
    indexImage += [imageID for _ in range(patchID)]
    
    # print (cnnFea.shape, len(indexImage))
    if imageID == 0:
      label_cnnFea = cnnFea
    else:
      label_cnnFea = np.concatenate((label_cnnFea, cnnFea), axis=1)
  with open(conf_imageDict, 'wb') as f:
    print("Writing new image Dict file")
    pickle.dump(image_dict, f)
  with open(conf_indexImg, 'wb') as f:
    print ("Writing new index Img file")
    pickle.dump(indexImage, f)
  with open(conf_cnnFeatures, 'wb') as f:
    print ("Writing npy file")
    np.save(f, label_cnnFea)

In [ ]:
def sample_images (im_path, patchSize):
  I = plt.imread(im_path)
  if I.dtype == np.uint8:
    I = I.astype('<f')
  # print (np.mean(I, (0,1)))
  imHeight, imWidth, imDepth = I.shape
  if imHeight > imWidth:
    new_ht = int((256/float(imWidth))*imHeight)
    I = resize(I, (new_ht, 256))
  else:
    new_width = int((256/float(imHeight)) * imWidth)
    I = resize(I, (256, new_width))
  
  patches = view_as_windows(I, (patchSize, patchSize,3), step=conf_stepSize)
  # print (patches.shape)
  return patches

In [ ]:
def debug_img (img, txt="", rgb=True):
  print (txt)
  if rgb:
    if np.all (img<256):
      plt.imshow(img/256)
    else:
      plt.imshow(img)
  else:
    temp = img[:,:,0]
    img[:,:,0] = img[:,:,2]
    img[:,:,2] = temp
    plt.imshow(img)
  plt.show()

In [ ]:
#Main loop for extracting CNN features and mining patterns

for i in range(1,conf_numClasses+1):
  print ('Extracting cnn features from class', i)
  cnnFeaExtraction(i, conf_patchSize)


Sample Code to extract features from a single image

In [ ]:
im_path = "/content/drive/My Drive/MDPM/data/shoes/image/127811_zappos_20181018_9124097_image0000.jpg"
 
net = setup_caffe()
patches = sample_images(im_path, conf_patchSize)

patchID = 0
for i in range(patches.shape[0]):
  for j in range(patches.shape[1]):
    patch = patches[i][j][0]
    patch = resize(patch, (CROPPED_DIM, CROPPED_DIM))
    patch = np.swapaxes(patch, 0, 1)
    
    patch_flat = np.sum(patch, axis=2)
    # print (pch.size, np.nonzero(pch-255)[0].size, int(0.2*pch.size))
    if np.nonzero(patch_flat-765.0)[0].size > int(0.2*patch_flat.size):
      out = caffe_net_forward(patch, net)
      # print (out.shape)
      if patchID == 0:
        cnnFea = out.T
      else:
        cnnFea = np.concatenate((cnnFea, out.T), axis=1)
      patchID += 1

# print (cnnFea.shape, len(indexImage))
label_cnnFea = cnnFea

with open(conf_cnnFeatures, 'wb') as f:
  print ("Writing npy file")
  np.save(f, label_cnnFea)

Writing npy file


Sample code to save patches as a dataset

In [ ]:
def save_patches (classId, patch_size):
  label = labels[classId-1]  
  for imageID, im_path in enumerate(label.iterdir()):
    patches = sample_images(im_path, patch_size)
    patchId = 1
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
          patch = patches[i][j][0]
          with open ('/content/drive/My Drive/MDPM/patches/'+ str(imageID+1) + "_" + str(patchId)+'.npy', 'wb') as f:
            np.save(f, patch)
          patchId += 1
  
save_patches(1, conf_patchSizeL3)

[161.15436 161.12009 168.11465]
[159.84453 171.8712  207.5988 ]


Code to get dataset mean for the shoes images


In [ ]:
def get_dataset_mean(classId, patchSize):
  label = labels[classId-1]  

  sum_arr = np.zeros((3,))
  pixel_num = 0
  num_images = 0
  
  for imageID, im_path in enumerate(label.iterdir()):
    num_images += 1
    patches = sample_images(im_path, patchSize)
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
          patch = patches[i][j][0]
          patch = patch/256
          sum_arr[0] += np.sum(patch[:,:,0])         
          sum_arr[1] += np.sum(patch[:,:,1])
          sum_arr[2] += np.sum(patch[:,:,2])
          pixel_num += np.size(patch[:,:,0])
  print ("Analyzed", num_images, "images")
  return (sum_arr/pixel_num)*256
  
IMAGE_MEAN = get_dataset_mean(1, conf_patchSize)
print ('RGB IMAGE_MEAN is', IMAGE_MEAN)
temp = IMAGE_MEAN[0]
IMAGE_MEAN[0] = IMAGE_MEAN[2]
IMAGE_MEAN[2] = temp
print ('BGR IMAGE_MEAN is', IMAGE_MEAN)

Analyzed 130 images
RGB IMAGE_MEAN is [169.37496185 158.50061805 158.12070422]
BGR IMAGE_MEAN is [158.12070422 158.50061805 169.37496185]


In [ ]:
IMAGE_MEAN

array([158.12070422, 158.50061805, 169.37496185])

In [ ]:
with open(conf_image_mean, 'wb') as f:
  np.save(f, IMAGE_MEAN)

Code to load ilvsvrc mean (BGR format)

In [ ]:
def get_ilsvrc_mean ():
  mean_arr = np.load(ROOT + 'ilsvrc_2012_mean.npy')
  assert (mean_arr.dtype == 'float64')
  #mean_arr = np.swapaxes(mean_arr,0,2)
  mean_arr = mean_arr.mean(1).mean(1)
  #plt.imshow(mean_arr.mean(1).mean(1))
  return mean_arr

#IMAGE_MEAN = get_ilsvrc_mean()

Combine the cnnFeature files

In [ ]:
import numpy as np

# cnn1 = '/content/drive/My Drive/MDPM/output/shoes/cnnFeatures1.npy'
# cnn2 = '/content/drive/My Drive/MDPM/output/shoes/cnnFeatures2.npy'
# cnn3 = '/content/drive/My Drive/MDPM/output/shoes/cnnFeatures3.npy'
# cnn4 = '/content/drive/My Drive/MDPM/output/shoes/cnnFeatures4.npy'
cnn5 = './MDPM_python/cnnFeatures5.npy'
# cnn12 = './MDPM_python/cnnFeatures12.npy'
# cnn34 = './MDPM_python/cnnFeatures34.npy'
cnn1234 = './MDPM_python/cnnFeatures1234.npy'
cnn = './MDPM_python/cnnFeatures.npy'

# cnnFeat1 = np.load(cnn1)
# cnnFeat2 = np.load(cnn2)
# cnnFeat3 = np.load(cnn3)
# cnnFeat4 = np.load(cnn4)
# cnnFeat5 = np.load(cnn5)

cnnFeat5 = np.load(cnn5)
# cnnFeat34 = np.load(cnn34)

In [ ]:
cnnFeat = np.concatenate((cnnFeat1234,cnnFeat5), axis=1)
np.save(cnn,cnnFeat)

In [ ]:
assert cnnFeat.shape[0] == cnnFeat1234.shape[0] == cnnFeat5.shape[0]# == cnnFeat3.shape[0] == cnnFeat4.shape[0] == cnnFeat5.shape[0]

In [ ]:
assert cnnFeat.shape[1] == (cnnFeat1234.shape[1] + cnnFeat5.shape[1] )#+ cnnFeat3.shape[1] + cnnFeat4.shape[1] + cnnFeat5.shape[1])

In [ ]:
assert np.all(cnnFeat[:,0] == cnnFeat12[:,0])
assert np.all(cnnFeat[:,205584] == cnnFeat5[:,0])